In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local[*]')\
        .appName('test1')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 06:15:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet('../notebooks/data/raw/green/2021/*')

In [ ]:
df_green.columns

In [5]:
df_green.createOrReplaceTempView('green')

In [6]:
df_result_green = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS revenue_zone,

    -- Revenue calculation 
    SUM(total_amount) AS revenue_monthly_total_amount,
    count(1) as count
FROM
    green
where lpep_pickup_datetime >= '2021-01-01 00:00:00'
GROUP BY
    1, 2
order by
    1,2
""")

In [7]:
df_result_green.show()

+-------------------+------------+----------------------------+-----+
|               hour|revenue_zone|revenue_monthly_total_amount|count|
+-------------------+------------+----------------------------+-----+
|2021-01-01 00:00:00|           7|                       61.47|    2|
|2021-01-01 00:00:00|          17|                      102.34|    3|
|2021-01-01 00:00:00|          35|                        50.3|    2|
|2021-01-01 00:00:00|          39|                        36.0|    1|
|2021-01-01 00:00:00|          41|                         8.3|    1|
|2021-01-01 00:00:00|          42|                       28.02|    3|
|2021-01-01 00:00:00|          43|                         6.8|    1|
|2021-01-01 00:00:00|          47|                       63.19|    2|
|2021-01-01 00:00:00|          55|                       57.25|    1|
|2021-01-01 00:00:00|          61|          43.760000000000005|    2|
|2021-01-01 00:00:00|          63|                       41.99|    1|
|2021-01-01 00:00:00

In [8]:
df_result_green\
.repartition(20)\
.write.parquet('../notebooks/data/report/revenue/green',mode = 'overwrite')

In [9]:
df_yellow = spark.read.parquet('../notebooks/data/raw/yellow/2021/*')

In [10]:
df_yellow = spark.read.parquet('../notebooks/data/raw/yellow/2021/*')

In [11]:
df_yellow.createOrReplaceTempView('yellow')

In [12]:
df_result_yellow = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS revenue_zone,

    -- Revenue calculation 
    SUM(total_amount) AS revenue_monthly_total_amount,
    count(1) as count
FROM
    yellow
where tpep_pickup_datetime >= '2021-01-01 00:00:00'
GROUP BY
    1, 2
order by
    1,2
""")

In [52]:
df_result_yellow.show()

+-------------------+------------+----------------------------+-----+
|               hour|revenue_zone|revenue_monthly_total_amount|count|
+-------------------+------------+----------------------------+-----+
|2021-01-01 00:00:00|           4|                        64.4|    3|
|2021-01-01 00:00:00|           7|                        71.3|    2|
|2021-01-01 00:00:00|          10|                        52.4|    1|
|2021-01-01 00:00:00|          13|                        73.9|    3|
|2021-01-01 00:00:00|          17|                        32.3|    2|
|2021-01-01 00:00:00|          24|                      120.54|    7|
|2021-01-01 00:00:00|          25|                       19.04|    1|
|2021-01-01 00:00:00|          28|                       44.16|    1|
|2021-01-01 00:00:00|          32|                       30.32|    1|
|2021-01-01 00:00:00|          41|           94.78999999999999|    5|
|2021-01-01 00:00:00|          42|          62.099999999999994|    2|
|2021-01-01 00:00:00

In [13]:
df_result_green_tmp =df_result_green\
    .withColumnRenamed('revenue_monthly_total_amount','green_amount')\
    .withColumnRenamed('count','green_count')

df_result_yellow_tmp =df_result_yellow\
    .withColumnRenamed('revenue_monthly_total_amount','yellow_amount')\
    .withColumnRenamed('count','yellow_count')

In [14]:
df_result_yellow\
.repartition(20)\
.write.parquet('../notebooks/data/report/revenue/yellow',mode = 'overwrite')

In [15]:
df_join = df_result_green_tmp.join(df_result_yellow_tmp, on=['hour','revenue_zone'], how='outer')

In [17]:
df_join.write.parquet('../notebooks/data/report/total',mode='overwrite')

In [18]:
df_join = spark.read.parquet('../notebooks/data/report/total')

In [19]:
df_join.show()

+-------------------+------------+------------+-----------+------------------+------------+
|               hour|revenue_zone|green_amount|green_count|     yellow_amount|yellow_count|
+-------------------+------------+------------+-----------+------------------+------------+
|2021-01-01 00:00:00|           7|       61.47|          2|              71.3|           2|
|2021-01-01 00:00:00|          17|      102.34|          3|              32.3|           2|
|2021-01-01 00:00:00|          32|        null|       null|             30.32|           1|
|2021-01-01 00:00:00|          39|        36.0|          1|              null|        null|
|2021-01-01 00:00:00|          45|        null|       null|             93.78|           4|
|2021-01-01 00:00:00|          49|        null|       null|            247.57|           6|
|2021-01-01 00:00:00|          51|        null|       null|             17.18|           1|
|2021-01-01 00:00:00|          63|       41.99|          1|              null|  

In [20]:
df_zones = spark.read.parquet('../notebooks/zones')

In [21]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df_result = df_join.join(df_zones, df_zones.LocationID == df_join.revenue_zone )

In [23]:
df_result.drop('LocationID').write.parquet('../notebooks/tmp/revenue_zones')